## Read the original Excel file and extract the relevant columns and rows

In [2]:
import pandas as pd

In [3]:
df = pd.read_excel('ALT_Übersicht_EU-Rechtsmonitoring.xlsx', sheet_name='Protokoll')

In [4]:
subset_df = df.iloc[0:]
subset_df.columns = df.iloc[0]
# remove the title rows that contains descriptions of column titles
subset_df = subset_df.drop(subset_df.index[0])
subset_df = subset_df.reset_index(drop=True)
subset_df = subset_df.drop(subset_df.index[0])
subset_df = subset_df.reset_index(drop=True)
subset_df = subset_df[['Fund?', 'Aktenzeichen', 'Kurzzusammenfassung (Onepager)']]
subset_df

,Fund?,Aktenzeichen,Kurzzusammenfassung (Onepager)
0,ja,C20407 Dokumentnr.: Ref. Ares(2022)7305537,Im Rahmen der Initiative sollen harmonisierte ...
1,ja,RL (EU) 2022/1999,Die Richtlinie soll die EU-Mitgliedsstaaten zu...
2,ja,VO (EU) 2022/2002,"Zwar lebensmittel-, bzw. produktrechtliche Det..."
3,ja,RL (EU) 2022/2041,Die Richtlinie soll insbesondere die Angemesse...
4,NaN,VO (EU) 2022/2046,Beibehaltung der bisher geltenden Regelungen z...
...,...,...,...
909,ja,https://eur-lex.europa.eu/legal-content/DE/TXT...,Anpassung von zwei Anforderungen an die von Un...
910,nein,NaN,NaN
911,nein,NaN,NaN
912,ja,Richtlinie - EU - 2025/25 - DE - EUR-Lex,Es handelt sich um eine an die Mitgliedsstaate...


### Derive requried data:
  1) if the entry is a relevant hit
  2) link to the legal act from EU on EUR-LEX

In [5]:
# only filter clear 'yes' cases from the list
subset_df['Fund?'].unique().tolist()

['ja',
 nan,
 'nein',
 'Ja',
 'ja / nein',
 'Nein',
 'ja ',
 'ja (?)',
 'nein ',
 'ja  ']

In [6]:
subset_df['Fund?'].value_counts()

Fund?
nein         391
ja           165
Ja            19
Nein           8
ja             6
ja             4
ja / nein      2
ja (?)         1
nein           1
Name: count, dtype: int64

In [7]:
subset_df['relevant'] = False
subset_df['relevant'] = subset_df['Fund?'].str.lower().str.strip() == 'ja'

### get the url for all legal acts, start with valid URLs first

In [8]:
# filter list to postiive hits and double check count
treffer_df = subset_df[subset_df['relevant'] == True]
len(treffer_df)

194

In [9]:
# check for valid urls first
from urllib.parse import urlparse
def get_url(string):
    try:
        result = urlparse(string.strip())
        if all([result.scheme, result.netloc]):
            return string
        else:
            return ''
    except:
        return ''

In [10]:
# retrieve all valid URLs and write them to a new column
subset_df['validated_url'] = subset_df['Aktenzeichen'].apply(get_url)

In [11]:
(subset_df['validated_url'] != '').sum()

np.int64(131)

In [19]:
def extract_oj_id(url):
    key = "uri=OJ:"
    idx = url.find(key)
    if idx != -1:
        return url[idx + len(key):]
    else:
        return ''

In [22]:
subset_df['oj_doc_id'] = subset_df['validated_url'].apply(extract_oj_id)
subset_df

,Fund?,Aktenzeichen,Kurzzusammenfassung (Onepager),relevant,validated_url,oj_doc_id
0,ja,C20407 Dokumentnr.: Ref. Ares(2022)7305537,Im Rahmen der Initiative sollen harmonisierte ...,True,,
1,ja,RL (EU) 2022/1999,Die Richtlinie soll die EU-Mitgliedsstaaten zu...,True,,
2,ja,VO (EU) 2022/2002,"Zwar lebensmittel-, bzw. produktrechtliche Det...",True,,
3,ja,RL (EU) 2022/2041,Die Richtlinie soll insbesondere die Angemesse...,True,,
4,NaN,VO (EU) 2022/2046,Beibehaltung der bisher geltenden Regelungen z...,False,,
...,...,...,...,...,...,...
909,ja,https://eur-lex.europa.eu/legal-content/DE/TXT...,Anpassung von zwei Anforderungen an die von Un...,True,https://eur-lex.europa.eu/legal-content/DE/TXT...,L_202590005
910,nein,NaN,NaN,False,,
911,nein,NaN,NaN,False,,
912,ja,Richtlinie - EU - 2025/25 - DE - EUR-Lex,Es handelt sich um eine an die Mitgliedsstaate...,True,,


### Write Result to disk

In [25]:
# write to disk
subset_df[subset_df['relevant']].to_excel('consolidated-legal-acts.xlsx', index=False)

,Fund?,Aktenzeichen,Kurzzusammenfassung (Onepager),relevant,validated_url,oj_doc_id
0,ja,C20407 Dokumentnr.: Ref. Ares(2022)7305537,Im Rahmen der Initiative sollen harmonisierte ...,True,,
1,ja,RL (EU) 2022/1999,Die Richtlinie soll die EU-Mitgliedsstaaten zu...,True,,
2,ja,VO (EU) 2022/2002,"Zwar lebensmittel-, bzw. produktrechtliche Det...",True,,
3,ja,RL (EU) 2022/2041,Die Richtlinie soll insbesondere die Angemesse...,True,,
4,NaN,VO (EU) 2022/2046,Beibehaltung der bisher geltenden Regelungen z...,False,,
...,...,...,...,...,...,...
909,ja,https://eur-lex.europa.eu/legal-content/DE/TXT...,Anpassung von zwei Anforderungen an die von Un...,True,https://eur-lex.europa.eu/legal-content/DE/TXT...,L_202590005
910,nein,NaN,NaN,False,,
911,nein,NaN,NaN,False,,
912,ja,Richtlinie - EU - 2025/25 - DE - EUR-Lex,Es handelt sich um eine an die Mitgliedsstaate...,True,,
